# Лабораторная работа №2. Предварительная обработка текстовых данных

## Цель работы
Получить практические навыки обработки текстовых данных в среде Jupiter Notebook. Научиться проводить предварительную обработку текстовых данных и выявлять параметры обработки, позволяющие добиться наилучшей точности классификации.

## Выполнение работы

### 1. В среде Jupyter Notebook создать новый ноутбук (Notebook)

### 2. Импортировать необходимые для работы библиотеки и модули


In [1]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer 
import numpy as np
import nltk
from nltk.stem import *
from nltk import word_tokenize
from sklearn.pipeline import Pipeline
from tqdm import tqdm
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report 
from sklearn.metrics import accuracy_score

### 3. Загрузить обучающую и экзаменационную выборку в соответствие с вариантом



In [2]:
categories = ['misc.forsale','sci.med','talk.religion.misc'] 
remove = ('headers', 'footers', 'quotes')
twenty_train = fetch_20newsgroups(subset='train', shuffle=True, random_state=23, categories = categories, remove = remove )
twenty_test = fetch_20newsgroups(subset='test', shuffle=True, random_state=23, categories = categories, remove = remove )

### 4. Вывести на экран по одному-два документа каждого класса.

In [3]:
print (twenty_train.target_names[twenty_train.target[1]])
print (twenty_train.target_names[twenty_train.target[2]])
print (twenty_train.target_names[twenty_train.target[7]])


misc.forsale
sci.med
talk.religion.misc


In [4]:
print(twenty_train.data[1])

One way ticket (return leg of roundtrip ticket) for female traveler



San Francisco ==> St. Louis ==> Philadelphia


May 21, 1993 (Friday)  leaves SFO     10:25 am
                       arrives Phila.  8:43 pm



.............$150   or best offer





In [5]:
print(twenty_train.data[2])

L(>  |JB>  1) Ron...what do YOU consider to be "proper channels"...
L(>  
L(>  |  I'm glad it caught your eye. That's the purpose of this forum to
L(>  | educate those, eager to learn, about the facts of life. That phrase
L(>  | is used to bridle the frenzy of all the would-be respondents, who
L(>  | otherwise would feel being left out as the proper authorities to be
L(>  | consulted on that topic. In short, it means absolutely nothing.
L(>  
L(>  An apt description of the content of just about all Ron Roth's 
L(>  posts to date.  At least there's entertainment value (though it 
L(>  is diminishing).

     Well, that's easy for *YOU* to say.  All *YOU* have to do is sit 
     back, soak it all in, try it out on your patients, and then brag
     to all your colleagues about that incredibly success rate you're
     having all of a sudden...

     --Ron--


In [6]:
print(twenty_train.data[7])



"Finally, brethern, whatever is true, whatever is honorable, whatever is
right, whatever is pure, whatever is lovely, whatever is of good repute,
if there is any excellence and if anything worthy of praise, let your
mind dwell on these things."  Phil. 4:8.


### 5. Применить стемминг, записав обработанные выборки (тестовую и обучающую) в новые переменные

In [7]:
porter_stemmer = PorterStemmer() 
def stemming(text: str) -> str:
  words = []
  for word in word_tokenize(text):
    words.append(porter_stemmer.stem(word.lower()))
  return ' '.join(words) 
def stemming_dataset(dataset: list) -> list: 
  resulting_list = []
  for example in tqdm(dataset): 
    resulting_list.append(stemming(example))
  return resulting_list 
stem_train = fetch_20newsgroups(subset='train', shuffle=True, random_state=23, categories = categories, remove = remove )
stem_test = fetch_20newsgroups(subset='test', shuffle=True, random_state=23, categories = categories, remove = remove )
stem_train['data_stem'] = stemming_dataset(stem_train['data']) 
stem_test['data_stem'] = stemming_dataset(stem_test['data']) 
del stem_train['data']
del stem_test['data']

100%|█████████████████████████████████████████████████████████████████████████████| 1037/1037 [00:08<00:00, 126.49it/s]


### 6. Провести векторизацию выборки: 
- Векторизовать обучающую и тестовую выборки простым подсчетом слов (CountVectorizer) 
- Вывести и проанализировать первые 20 наиболее частотных слов всей выборки и каждого класса по-отдельности. 
- Рассчитать сходство по коэффициенту Жаккара между тремя классами 
- Применить процедуру отсечения стоп-слов и повторить пункты 2,3. 
- Провести пункты 1-3 для обучающей и тестовой выборки, для которой проведена процедура стемминга. 
- Векторизовать выборки с помощью TfidfTransformer (с использованием TF и TF-IDF взвешиваний).

In [8]:
def jaccard_similarity(list1, list2):
    intersection = len(list(set(list1).intersection(list2)))
    union = (len(set(list1)) + len(set(list2))) - intersection
    return float(intersection) / union
def SortbyTF(inputStr):
    return inputStr[1]

In [9]:
vect = CountVectorizer(max_features = 10000)
vect.fit(twenty_train.data)

train_data = vect.transform(twenty_train.data)
test_data = vect.transform(twenty_test.data)

train_vec = list(zip(vect.get_feature_names_out(), np.ravel(train_data.sum(axis=0))))
test_vec = list(zip(vect.get_feature_names_out(), np.ravel(test_data.sum(axis=0))))

train_vec.sort(key=SortbyTF, reverse = True)
test_vec.sort(key=SortbyTF, reverse = True)
print (train_vec[:20],'\n')
print (test_vec[:20])

[('the', 11301), ('of', 6613), ('to', 6208), ('and', 5710), ('in', 3962), ('is', 3857), ('that', 3485), ('it', 2943), ('for', 2894), ('you', 2402), ('this', 1766), ('are', 1753), ('with', 1736), ('not', 1711), ('have', 1632), ('be', 1555), ('or', 1504), ('as', 1433), ('on', 1314), ('but', 1143)] 

[('the', 7706), ('of', 4314), ('to', 4227), ('and', 3922), ('in', 2670), ('is', 2596), ('that', 2302), ('for', 2017), ('it', 1819), ('you', 1541), ('have', 1230), ('with', 1157), ('are', 1149), ('this', 1149), ('not', 1084), ('or', 1009), ('be', 1002), ('as', 932), ('on', 926), ('if', 790)]


In [10]:
class_name = {i: twenty_train['target_names'][i] for i in range(len(twenty_train['target_names']))}
for i in range(3):
  twenty_train[class_name[i]] = [twenty_train['data'][j] for j in range(len(twenty_train['data']))\
                                 if twenty_train['target'][j] == i]

**Наиболее частые слова в классе `forsale`**

In [11]:
train_data_1 = vect.transform(twenty_train[class_name[0]]) 
x = list(zip(vect.get_feature_names(), np.ravel(train_data_1.sum(axis=0))))
list1=x
x.sort(key=SortbyTF, reverse = True) 
print (x[:20])

[('the', 1488), ('for', 1157), ('and', 1081), ('to', 967), ('of', 755), ('in', 645), ('00', 617), ('it', 513), ('is', 510), ('you', 477), ('with', 452), ('or', 430), ('have', 378), ('are', 308), ('all', 294), ('if', 283), ('new', 278), ('this', 274), ('that', 249), ('sale', 247)]


C:\Users\yaasd\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


**Наиболее частые слова в классе `med`**

In [12]:
train_data_2 = vect.transform(twenty_train[class_name[1]]) 
x = list(zip(vect.get_feature_names(), np.ravel(train_data_2.sum(axis=0))))
list2=x
x.sort(key=SortbyTF, reverse = True) 
print (x[:20])

[('the', 5174), ('of', 3283), ('to', 2879), ('and', 2643), ('in', 1971), ('is', 1899), ('that', 1526), ('it', 1467), ('for', 1086), ('you', 863), ('this', 852), ('are', 812), ('be', 787), ('with', 776), ('not', 730), ('have', 700), ('or', 648), ('on', 635), ('as', 624), ('but', 530)]


**Наиболее частые слова в классе `religion`**

In [13]:
train_data_3 = vect.transform(twenty_train[class_name[2]]) 
x = list(zip(vect.get_feature_names(), np.ravel(train_data_3.sum(axis=0))))
list3=x
x.sort(key=SortbyTF, reverse = True) 
print (x[:20])

[('the', 4639), ('of', 2575), ('to', 2362), ('and', 1986), ('that', 1710), ('is', 1448), ('in', 1346), ('you', 1062), ('it', 963), ('not', 785), ('for', 651), ('as', 642), ('this', 640), ('are', 633), ('be', 573), ('have', 554), ('with', 508), ('was', 486), ('he', 470), ('they', 445)]


**Рассчёт сходства по коэффициенту Жаккара между тремя классами**

In [14]:
print ("Коэффициент сходства между классом 0 и 1: ", jaccard_similarity(list1, list2))
print ("Коэффициент сходства между классом 1 и 2: ", jaccard_similarity(list2, list3))
print ("Коэффициент сходства между классом 0 и 2: ", jaccard_similarity(list1, list3))

Коэффициент сходства между классом 0 и 1:  0.07764426962659626
Коэффициент сходства между классом 1 и 2:  0.0949904188338352
Коэффициент сходства между классом 0 и 2:  0.1367511651699443


**Применение процедуры отсечения стоп-слов**

In [15]:
vect = CountVectorizer(max_features = 10000, stop_words = 'english') 
vect.fit(twenty_train.data)
train_data_4 = vect.transform(twenty_train.data) 
test_data_4 = vect.transform(twenty_test.data)

**Вывод наиболее частых слов для всей выборки**

In [16]:
x = list(zip(vect.get_feature_names(), np.ravel(train_data_4.sum(axis=0))))
x.sort(key=SortbyTF, reverse = True) 
print (x[:20])

[('00', 640), ('people', 517), ('new', 504), ('edu', 502), ('don', 467), ('like', 461), ('good', 420), ('just', 417), ('know', 394), ('10', 358), ('use', 356), ('god', 338), ('time', 336), ('think', 328), ('does', 313), ('20', 285), ('used', 275), ('50', 261), ('com', 259), ('jesus', 258)]


**Наиболее частые слова в классе `forsale`**

In [17]:
train_data_1 = vect.transform(twenty_train[class_name[0]]) 
x = list(zip(vect.get_feature_names(), np.ravel(train_data_1.sum(axis=0))))
list1=x
x.sort(key=SortbyTF, reverse = True) 
print (x[:20])

[('00', 617), ('new', 278), ('sale', 247), ('50', 228), ('10', 214), ('dos', 201), ('offer', 195), ('shipping', 180), ('20', 168), ('price', 164), ('25', 155), ('15', 153), ('condition', 151), ('good', 146), ('used', 130), ('like', 129), ('edu', 128), ('asking', 121), ('mail', 121), ('interested', 120)]


**Наиболее частые слова в классе `med`**

In [18]:
train_data_2 = vect.transform(twenty_train[class_name[1]]) 
x = list(zip(vect.get_feature_names(), np.ravel(train_data_2.sum(axis=0))))
list2=x
x.sort(key=SortbyTF, reverse = True) 
print (x[:20])

[('edu', 354), ('don', 230), ('people', 221), ('health', 217), ('use', 216), ('medical', 206), ('like', 201), ('know', 191), ('com', 190), ('time', 180), ('just', 174), ('patients', 163), ('new', 162), ('think', 153), ('disease', 146), ('good', 143), ('msg', 143), ('food', 142), ('years', 139), ('doctor', 133)]


**Наиболее частые слова в классе `religion`**

In [19]:
train_data_3 = vect.transform(twenty_train[class_name[2]]) 
x = list(zip(vect.get_feature_names(), np.ravel(train_data_3.sum(axis=0))))
list3=x
x.sort(key=SortbyTF, reverse = True) 
print (x[:20])

[('god', 329), ('people', 267), ('jesus', 256), ('don', 162), ('bible', 160), ('just', 159), ('christian', 151), ('think', 151), ('know', 149), ('say', 149), ('does', 147), ('did', 132), ('good', 131), ('like', 131), ('life', 118), ('way', 118), ('believe', 117), ('said', 103), ('point', 101), ('time', 99)]


**Рассчёт сходства по коэффициенту Жаккара между тремя классами**

In [20]:
print ("Коэффициент сходства между классом 0 и 1: ", jaccard_similarity(list1, list2))
print ("Коэффициент сходства между классом 1 и 2: ", jaccard_similarity(list2, list3))
print ("Коэффициент сходства между классом 0 и 2: ", jaccard_similarity(list1, list3))

Коэффициент сходства между классом 0 и 1:  0.08166576527852894
Коэффициент сходства между классом 1 и 2:  0.10083663584324086
Коэффициент сходства между классом 0 и 2:  0.14311842706904435


**Повторение после применения стемминга**

In [21]:
vect = CountVectorizer(max_features = 10000) 
vect.fit(stem_train.data_stem)
train_data_stem = vect.transform(stem_train.data_stem) 
test_data_stem = vect.transform(stem_test.data_stem)

In [22]:
x = list(zip(vect.get_feature_names(), np.ravel(train_data_stem.sum(axis=0))))
x.sort(key=SortbyTF, reverse = True) 
print (x[:20])

[('the', 11298), ('of', 6613), ('to', 6208), ('and', 5712), ('in', 3964), ('is', 3922), ('that', 3488), ('it', 3111), ('for', 2894), ('you', 2401), ('are', 1786), ('not', 1780), ('have', 1774), ('thi', 1770), ('be', 1764), ('with', 1737), ('or', 1504), ('as', 1431), ('do', 1386), ('on', 1320)]


In [23]:
class_name = {i: stem_train['target_names'][i] for i in range (len(stem_train['target_names']))}
for i in range(3):
  stem_train[class_name[i]] = [stem_train['data_stem'][j] for j in range(len(stem_train['data_stem']))\
                               if stem_train['target'][j] == i]

**Наиболее частые слова в классе `forsale` при стемминге**

In [24]:
train_data_stem_1 = vect.transform(stem_train[class_name[0]]) 
x = list(zip(vect.get_feature_names(), np.ravel(train_data_stem_1.sum(axis=0))))
list1 = x
x.sort(key=SortbyTF, reverse = True) 
print (x[:20])

[('the', 1486), ('for', 1157), ('and', 1081), ('to', 967), ('of', 755), ('in', 647), ('00', 617), ('it', 532), ('is', 516), ('you', 477), ('with', 453), ('or', 430), ('have', 385), ('do', 318), ('are', 310), ('all', 292), ('if', 283), ('new', 278), ('thi', 274), ('will', 253)]


**Наиболее частые слова в классе `med` при стемминге**

In [25]:
train_data_stem_2 = vect.transform(stem_train[class_name[1]]) 
x = list(zip(vect.get_feature_names(), np.ravel(train_data_stem_2.sum(axis=0))))
list2 = x
x.sort(key=SortbyTF, reverse = True) 
print (x[:20])

[('the', 5173), ('of', 3283), ('to', 2879), ('and', 2644), ('in', 1971), ('is', 1936), ('it', 1539), ('that', 1527), ('for', 1086), ('be', 865), ('you', 863), ('thi', 853), ('are', 828), ('have', 786), ('with', 776), ('not', 749), ('or', 648), ('on', 637), ('as', 624), ('do', 552)]


**Наиболее частые слова в классе `religion` при стемминге**

In [26]:
train_data_stem_3 = vect.transform(stem_train[class_name[2]]) 
x = list(zip(vect.get_feature_names(), np.ravel(train_data_stem_3.sum(axis=0))))
list3 = x
x.sort(key=SortbyTF, reverse = True) 
print (x[:20])

[('the', 4639), ('of', 2575), ('to', 2362), ('and', 1987), ('that', 1711), ('is', 1470), ('in', 1346), ('you', 1061), ('it', 1040), ('not', 829), ('be', 696), ('for', 651), ('are', 648), ('thi', 643), ('as', 640), ('have', 603), ('do', 516), ('with', 508), ('wa', 493), ('he', 470)]


**Рассчёт сходства по коэффициенту Жаккара между тремя классами при стемминге**

In [27]:
print ("Коэффициент сходства между классом 0 и 1: ", jaccard_similarity(list1, list2))
print ("Коэффициент сходства между классом 1 и 2: ", jaccard_similarity(list2, list3))
print ("Коэффициент сходства между классом 0 и 2: ", jaccard_similarity(list1, list3))

Коэффициент сходства между классом 0 и 1:  0.08902804247209366
Коэффициент сходства между классом 1 и 2:  0.113895850737956
Коэффициент сходства между классом 0 и 2:  0.15620302925193663


**Векторизовать выборки с помощью TfidfTransformer (с использованием TF и TF-IDF взвешиваний).**

In [28]:
tfidf = TfidfTransformer(use_idf = True).fit(train_data)
train_data_tfidf = tfidf.transform(train_data)

### 7. Используя конвейер (Pipeline) реализовать модель Наивного Байесовского классификатора и выявить на основе показателей качества (значения полноты, точности, f1-меры и аккуратности), какая предварительная обработка данных обеспечит наилучшие результаты классификации. Должны быть исследованы следующие характеристики:

- Наличие \ отсутствие стемминга
- Отсечение \ не отсечение стоп-слов
- Взвешивание: Count, TF, TF-IDF
- Количество информативных терминов (max_features) - исследовать 5 значений в диапазоне от 100 до общего количества слов в выборке.



**Взвешивание Count при отсутствии стемминга и отсечения стоп-слов; число информативных признаков 10000**

In [29]:
text_clf = Pipeline([('vect', CountVectorizer(max_features= 10000)), ('clf', MultinomialNB ()),])
text_clf = text_clf.fit(twenty_train.data, twenty_train.target) 
prediction = text_clf.predict(twenty_test.data)
print('Accuracy score: ', accuracy_score(prediction, twenty_test.target)) 
print(classification_report(twenty_test.target, prediction))

Accuracy score:  0.90549662487946
              precision    recall  f1-score   support

           0       0.99      0.91      0.94       390
           1       0.89      0.91      0.90       396
           2       0.82      0.90      0.86       251

    accuracy                           0.91      1037
   macro avg       0.90      0.90      0.90      1037
weighted avg       0.91      0.91      0.91      1037



**Взвешивание Count при наличии стемминга и отсутствии отсечения стоп-слов; число информативных признаков 10000**

In [30]:
text_clf = Pipeline([('vect', CountVectorizer(max_features= 10000)), ('clf', MultinomialNB ()),])
text_clf = text_clf.fit(stem_train.data_stem, stem_train.target) 
prediction = text_clf.predict(stem_test.data_stem)
print('Accuracy score: ', accuracy_score(prediction, stem_test.target)) 
print(classification_report(stem_test.target, prediction))

Accuracy score:  0.9112825458052073
              precision    recall  f1-score   support

           0       0.99      0.92      0.95       390
           1       0.89      0.92      0.91       396
           2       0.84      0.88      0.86       251

    accuracy                           0.91      1037
   macro avg       0.91      0.91      0.91      1037
weighted avg       0.91      0.91      0.91      1037



**Взвешивание Count при наличии стемминга и отсечения стоп-слов; число информативных признаков 10000**

In [31]:
text_clf = Pipeline([('vect', CountVectorizer(max_features= 10000, stop_words = 'english')), ('clf', MultinomialNB ()),])
text_clf = text_clf.fit(stem_train.data_stem, stem_train.target) 
prediction = text_clf.predict(stem_test.data_stem)
print('Accuracy score: ', accuracy_score(prediction, stem_test.target)) 
print(classification_report(stem_test.target, prediction))

Accuracy score:  0.9170684667309547
              precision    recall  f1-score   support

           0       0.97      0.93      0.95       390
           1       0.90      0.93      0.91       396
           2       0.87      0.88      0.88       251

    accuracy                           0.92      1037
   macro avg       0.91      0.91      0.91      1037
weighted avg       0.92      0.92      0.92      1037



**Взвешивание Count при наличии стемминга и отсечения стоп-слов; число информативных признаков 100**

In [32]:
text_clf = Pipeline([('vect', CountVectorizer(max_features= 100, stop_words = 'english')), ('clf', MultinomialNB ()),])
text_clf = text_clf.fit(stem_train.data_stem, stem_train.target) 
prediction = text_clf.predict(stem_test.data_stem)
print('Accuracy score: ', accuracy_score(prediction, stem_test.target)) 
print(classification_report(stem_test.target, prediction))

Accuracy score:  0.7984570877531341
              precision    recall  f1-score   support

           0       0.91      0.84      0.87       390
           1       0.74      0.83      0.78       396
           2       0.74      0.69      0.71       251

    accuracy                           0.80      1037
   macro avg       0.80      0.79      0.79      1037
weighted avg       0.80      0.80      0.80      1037



**Взвешивание Count при наличии стемминга и отсечения стоп-слов; число информативных признаков 1000**

In [33]:
text_clf = Pipeline([('vect', CountVectorizer(max_features= 1000, stop_words = 'english')), ('clf', MultinomialNB ()),])
text_clf = text_clf.fit(stem_train.data_stem, stem_train.target) 
prediction = text_clf.predict(stem_test.data_stem)
print('Accuracy score: ', accuracy_score(prediction, stem_test.target)) 
print(classification_report(stem_test.target, prediction))

Accuracy score:  0.875602700096432
              precision    recall  f1-score   support

           0       0.94      0.91      0.93       390
           1       0.86      0.86      0.86       396
           2       0.80      0.84      0.82       251

    accuracy                           0.88      1037
   macro avg       0.87      0.87      0.87      1037
weighted avg       0.88      0.88      0.88      1037



**Взвешивание Count при наличии стемминга и отсечения стоп-слов; число информативных признаков 5000**

In [34]:
text_clf = Pipeline([('vect', CountVectorizer(max_features= 5000, stop_words = 'english')), ('clf', MultinomialNB ()),])
text_clf = text_clf.fit(stem_train.data_stem, stem_train.target) 
prediction = text_clf.predict(stem_test.data_stem)
print('Accuracy score: ', accuracy_score(prediction, stem_test.target)) 
print(classification_report(stem_test.target, prediction))

Accuracy score:  0.9132111861137898
              precision    recall  f1-score   support

           0       0.96      0.93      0.95       390
           1       0.89      0.92      0.91       396
           2       0.87      0.88      0.87       251

    accuracy                           0.91      1037
   macro avg       0.91      0.91      0.91      1037
weighted avg       0.91      0.91      0.91      1037



**Взвешивание TF-IDF при отсутствии стемминга и отсечения стоп-слов; число информативных признаков 10000**

In [35]:
text_clf = Pipeline([('vect', CountVectorizer(max_features= 10000)), 
                     ('tfidf', TfidfTransformer(use_idf = True)), 
                     ('clf', MultinomialNB ()),])
text_clf = text_clf.fit(twenty_train.data, twenty_train.target) 
prediction = text_clf.predict(twenty_test.data)
print('Accuracy score: ', accuracy_score(prediction, twenty_test.target)) 
print(classification_report(twenty_test.target, prediction))

Accuracy score:  0.8187078109932497
              precision    recall  f1-score   support

           0       0.97      0.91      0.94       390
           1       0.69      0.97      0.81       396
           2       0.95      0.44      0.60       251

    accuracy                           0.82      1037
   macro avg       0.87      0.77      0.78      1037
weighted avg       0.86      0.82      0.81      1037



**Взвешивание TF-IDF при наличии стемминга и отсутствии отсечения стоп-слов; число информативных признаков 10000**

In [36]:
text_clf = Pipeline([('vect', CountVectorizer(max_features= 10000)), 
                     ('tfidf', TfidfTransformer(use_idf = True)), 
                     ('clf', MultinomialNB ()),])
text_clf = text_clf.fit(stem_train.data_stem, stem_train.target) 
prediction = text_clf.predict(stem_test.data_stem)
print('Accuracy score: ', accuracy_score(prediction, stem_test.target)) 
print(classification_report(stem_test.target, prediction))

Accuracy score:  0.8177434908389586
              precision    recall  f1-score   support

           0       0.99      0.91      0.95       390
           1       0.69      0.98      0.81       396
           2       0.95      0.41      0.57       251

    accuracy                           0.82      1037
   macro avg       0.88      0.77      0.78      1037
weighted avg       0.86      0.82      0.80      1037



**Взвешивание TF-IDF при наличии стемминга и отсечения стоп-слов; число информативных признаков 10000**

In [37]:
text_clf = Pipeline([('vect', CountVectorizer(max_features= 10000, stop_words = 'english')), 
                     ('tfidf', TfidfTransformer(use_idf = True)), 
                     ('clf', MultinomialNB ()),])
text_clf = text_clf.fit(stem_train.data_stem, stem_train.target) 
prediction = text_clf.predict(stem_test.data_stem)
print('Accuracy score: ', accuracy_score(prediction, stem_test.target)) 
print(classification_report(stem_test.target, prediction))

Accuracy score:  0.8736740597878495
              precision    recall  f1-score   support

           0       0.95      0.94      0.94       390
           1       0.79      0.96      0.86       396
           2       0.95      0.64      0.76       251

    accuracy                           0.87      1037
   macro avg       0.90      0.85      0.86      1037
weighted avg       0.89      0.87      0.87      1037



**Взвешивание TF-IDF при наличии стемминга и отсечения стоп-слов; число информативных признаков 100**

In [38]:
text_clf = Pipeline([('vect', CountVectorizer(max_features= 100, stop_words = 'english')), 
                     ('tfidf', TfidfTransformer(use_idf = True)), 
                     ('clf', MultinomialNB ()),])
text_clf = text_clf.fit(stem_train.data_stem, stem_train.target) 
prediction = text_clf.predict(stem_test.data_stem)
print('Accuracy score: ', accuracy_score(prediction, stem_test.target)) 
print(classification_report(stem_test.target, prediction))

Accuracy score:  0.7955641272902604
              precision    recall  f1-score   support

           0       0.90      0.85      0.87       390
           1       0.71      0.88      0.79       396
           2       0.82      0.57      0.67       251

    accuracy                           0.80      1037
   macro avg       0.81      0.77      0.78      1037
weighted avg       0.81      0.80      0.79      1037



**Взвешивание TF-IDF при наличии стемминга и отсечения стоп-слов; число информативных признаков 1000**

In [39]:
text_clf = Pipeline([('vect', CountVectorizer(max_features= 1000, stop_words = 'english')), 
                     ('tfidf', TfidfTransformer(use_idf = True)), 
                     ('clf', MultinomialNB ()),])
text_clf = text_clf.fit(stem_train.data_stem, stem_train.target) 
prediction = text_clf.predict(stem_test.data_stem)
print('Accuracy score: ', accuracy_score(prediction, stem_test.target)) 
print(classification_report(stem_test.target, prediction))

Accuracy score:  0.8727097396335584
              precision    recall  f1-score   support

           0       0.92      0.92      0.92       390
           1       0.83      0.91      0.87       396
           2       0.89      0.74      0.80       251

    accuracy                           0.87      1037
   macro avg       0.88      0.86      0.86      1037
weighted avg       0.87      0.87      0.87      1037



**Взвешивание TF-IDF при наличии стемминга и отсечения стоп-слов; число информативных признаков 5000**

In [40]:
text_clf = Pipeline([('vect', CountVectorizer(max_features= 5000, stop_words = 'english')), 
                     ('tfidf', TfidfTransformer(use_idf = True)), 
                     ('clf', MultinomialNB ()),])
text_clf = text_clf.fit(stem_train.data_stem, stem_train.target) 
prediction = text_clf.predict(stem_test.data_stem)
print('Accuracy score: ', accuracy_score(prediction, stem_test.target)) 
print(classification_report(stem_test.target, prediction))

Accuracy score:  0.8823529411764706
              precision    recall  f1-score   support

           0       0.94      0.95      0.94       390
           1       0.81      0.94      0.87       396
           2       0.95      0.69      0.79       251

    accuracy                           0.88      1037
   macro avg       0.90      0.86      0.87      1037
weighted avg       0.89      0.88      0.88      1037



**Взвешивание TF при отсутствии стемминга и отсечения стоп-слов; число информативных признаков 10000**

In [41]:
text_clf = Pipeline([('vect', CountVectorizer(max_features= 10000)), 
                     ('tfidf', TfidfTransformer(use_idf = False)), 
                     ('clf', MultinomialNB ()),])
text_clf = text_clf.fit(twenty_train.data, twenty_train.target) 
prediction = text_clf.predict(twenty_test.data)
print('Accuracy score: ', accuracy_score(prediction, twenty_test.target)) 
print(classification_report(twenty_test.target, prediction))

Accuracy score:  0.712632594021215
              precision    recall  f1-score   support

           0       0.98      0.87      0.92       390
           1       0.57      0.99      0.73       396
           2       1.00      0.04      0.08       251

    accuracy                           0.71      1037
   macro avg       0.85      0.63      0.57      1037
weighted avg       0.83      0.71      0.64      1037



**Взвешивание TF при наличии стемминга и отсутствии отсечения стоп-слов; число информативных признаков 10000**

In [42]:
text_clf = Pipeline([('vect', CountVectorizer(max_features= 10000)), 
                     ('tfidf', TfidfTransformer(use_idf = False)), 
                     ('clf', MultinomialNB ()),])
text_clf = text_clf.fit(stem_train.data_stem, stem_train.target) 
prediction = text_clf.predict(stem_test.data_stem)
print('Accuracy score: ', accuracy_score(prediction, stem_test.target)) 
print(classification_report(stem_test.target, prediction))

Accuracy score:  0.7184185149469624
              precision    recall  f1-score   support

           0       0.98      0.87      0.93       390
           1       0.58      0.99      0.73       396
           2       1.00      0.05      0.09       251

    accuracy                           0.72      1037
   macro avg       0.85      0.64      0.58      1037
weighted avg       0.83      0.72      0.65      1037



**Взвешивание TF-IDF при наличии стемминга и отсечения стоп-слов; число информативных признаков 10000**

In [43]:
text_clf = Pipeline([('vect', CountVectorizer(max_features= 10000, stop_words = 'english')), 
                     ('tfidf', TfidfTransformer(use_idf = False)), 
                     ('clf', MultinomialNB ()),])
text_clf = text_clf.fit(stem_train.data_stem, stem_train.target) 
prediction = text_clf.predict(stem_test.data_stem)
print('Accuracy score: ', accuracy_score(prediction, stem_test.target)) 
print(classification_report(stem_test.target, prediction))

Accuracy score:  0.8505303760848602
              precision    recall  f1-score   support

           0       0.94      0.93      0.93       390
           1       0.75      0.96      0.84       396
           2       0.96      0.57      0.71       251

    accuracy                           0.85      1037
   macro avg       0.88      0.82      0.83      1037
weighted avg       0.87      0.85      0.84      1037



**Взвешивание TF-IDF при наличии стемминга и отсечения стоп-слов; число информативных признаков 100**

In [44]:
text_clf = Pipeline([('vect', CountVectorizer(max_features= 100, stop_words = 'english')), 
                     ('tfidf', TfidfTransformer(use_idf = False)), 
                     ('clf', MultinomialNB ()),])
text_clf = text_clf.fit(stem_train.data_stem, stem_train.target) 
prediction = text_clf.predict(stem_test.data_stem)
print('Accuracy score: ', accuracy_score(prediction, stem_test.target)) 
print(classification_report(stem_test.target, prediction))

Accuracy score:  0.7888138862102217
              precision    recall  f1-score   support

           0       0.90      0.85      0.87       390
           1       0.70      0.89      0.78       396
           2       0.82      0.54      0.65       251

    accuracy                           0.79      1037
   macro avg       0.81      0.76      0.77      1037
weighted avg       0.80      0.79      0.78      1037



**Взвешивание TF-IDF при наличии стемминга и отсечения стоп-слов; число информативных признаков 1000**

In [45]:
text_clf = Pipeline([('vect', CountVectorizer(max_features= 1000, stop_words = 'english')), 
                     ('tfidf', TfidfTransformer(use_idf = False)), 
                     ('clf', MultinomialNB ()),])
text_clf = text_clf.fit(stem_train.data_stem, stem_train.target) 
prediction = text_clf.predict(stem_test.data_stem)
print('Accuracy score: ', accuracy_score(prediction, stem_test.target)) 
print(classification_report(stem_test.target, prediction))

Accuracy score:  0.8601735776277725
              precision    recall  f1-score   support

           0       0.91      0.92      0.91       390
           1       0.81      0.89      0.85       396
           2       0.88      0.72      0.79       251

    accuracy                           0.86      1037
   macro avg       0.87      0.84      0.85      1037
weighted avg       0.86      0.86      0.86      1037



**Взвешивание TF-IDF при наличии стемминга и отсечения стоп-слов; число информативных признаков 5000**

In [47]:
text_clf = Pipeline([('vect', CountVectorizer(stop_words = 'english')), 
                     ('tfidf', TfidfTransformer(use_idf = True)), 
                     ('clf', MultinomialNB ()),])
text_clf = text_clf.fit(stem_train.data_stem, stem_train.target) 
prediction = text_clf.predict(stem_test.data_stem)
print('Accuracy score: ', accuracy_score(prediction, stem_test.target)) 
print(classification_report(stem_test.target, prediction))

Accuracy score:  0.8688524590163934
              precision    recall  f1-score   support

           0       0.96      0.94      0.95       390
           1       0.77      0.97      0.86       396
           2       0.96      0.59      0.73       251

    accuracy                           0.87      1037
   macro avg       0.90      0.84      0.85      1037
weighted avg       0.89      0.87      0.86      1037



### 8. По результатам классификации занести в отчет выводы о влиянии каждого из этапов предобработки данных (наличие стемминга, взвешивание терминов, стоп-слова, количество информативных терминов) и о наиболее подходящей их комбинации. Объяснить различия (если имеются) в качестве классификации разных классов.

| Взвешивание | Стемминг | Отсечение стоп-слов | Число информативных признаков | Accuracy |      | Precision |      |      | Recall |      |      | f1-score |      |
|:-----------:|:--------:|:-------------------:|:-----------------------------:|:--------:|:----:|:---------:|:----:|:----:|:------:|:----:|:----:|:--------:|:----:|
|             |          |                     |                               |          |   0  |     1     |   2  |   0  |    1   |   2  |   0  |     1    |   2  |
|    Count    |     -    |          -          |             10000             |   0.905  | 0.99 |    0.89   | 0.82 | 0.91 |  0.91  |  0.9 | 0.94 |    0.9   | 0.86 |
|    Count    |     +    |          -          |             10000             |   0.911  | 0.99 |    0.89   | 0.84 | 0.92 |  0.92  | 0.88 | 0.95 |   0.91   | 0.86 |
|    Count    |     +    |          +          |              100              |   0.798  | 0.91 |    0.74   | 0.74 | 0.84 |  0.83  | 0.69 | 0.87 |   0.78   | 0.71 |
|    Count    |     +    |          +          |              1000             |   0.876  | 0.94 |    0.86   |  0.8 | 0.91 |  0.86  | 0.84 | 0.93 |   0.86   | 0.82 |
|    Count    |     +    |          +          |              5000             |   0.913  | 0.96 |    0.89   | 0.87 | 0.93 |  0.92  | 0.88 | 0.95 |   0.91   | 0.87 |
|    Count    |     +    |          +          |             10000             |   0.917  | 0.97 |    0.9    | 0.87 | 0.93 |  0.93  | 0.88 | 0.95 |   0.91   | 0.88 |
|    TF-IDF   |     -    |          -          |             10000             |   0.819  | 0.97 |    0.69   | 0.95 | 0.91 |  0.97  | 0.44 | 0.94 |   0.81   |  0.6 |
|    TF-IDF   |     +    |          -          |             10000             |   0.818  | 0.99 |    0.69   | 0.95 | 0.91 |  0.98  | 0.41 | 0.95 |   0.81   | 0.57 |
|    TF-IDF   |     +    |          +          |              100              |   0.796  |  0.9 |    0.71   | 0.82 | 0.85 |  0.88  | 0.57 | 0.87 |   0.79   | 0.67 |
|    TF-IDF   |     +    |          +          |              1000             |   0.873  | 0.92 |    0.83   | 0.89 | 0.92 |  0.91  | 0.74 | 0.92 |   0.87   |  0.8 |
|    TF-IDF   |     +    |          +          |              5000             |   0.882  | 0.94 |    0.81   | 0.95 | 0.95 |  0.94  | 0.69 | 0.94 |   0.87   | 0.79 |
|    TF-IDF   |     +    |          +          |             10000             |   0.874  | 0.95 |    0.79   | 0.95 | 0.94 |  0.96  | 0.64 | 0.94 |   0.86   | 0.76 |
|      TF     |     -    |          -          |             10000             |   0.713  | 0.98 |    0.57   |   1  | 0.87 |  0.99  | 0.04 | 0.92 |   0.73   | 0.08 |
|      TF     |     +    |          -          |             10000             |   0.718  | 0.98 |    0.58   |   1  | 0.87 |  0.99  | 0.05 | 0.93 |   0.73   | 0.09 |
|      TF     |     +    |          +          |              100              |   0.789  |  0.9 |    0.7    | 0.82 | 0.85 |  0.89  | 0.54 | 0.87 |   0.78   | 0.65 |
|      TF     |     +    |          +          |              1000             |   0.86   | 0.91 |    0.81   | 0.88 | 0.92 |  0.89  | 0.72 | 0.91 |   0.85   | 0.79 |
|      TF     |     +    |          +          |              5000             |   0.88   | 0.94 |    0.81   | 0.95 | 0.95 |  0.94  | 0.69 | 0.94 |   0.87   | 0.79 |
|      TF     |     +    |          +          |             10000             |   0.85   | 0.94 |    0.75   | 0.96 | 0.93 |  0.96  | 0.57 | 0.93 |   0.84   | 0.71 |